Create a csv file of the average metagenome abundance and weighted O2 consumption for each GEN in 20181030, 20190402, and 20190709 as well as the three averaged together. Also generate a csv file of the predicted respiration rate for all of my 10 cell GENs in 2018 and 2019 only

### Inputs
/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/GoM_mg_abundance_weighted_average_O2_consumption_per_cell.csv

/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/210121_summary_tables/GoM_metagenome_calculated_GEN_cells_per_ml.csv

/mnt/scgc/simon/microg2p/analyses/GoM_Summary_files/All_GoM_SAGs_1cell_20kb_decon_clusters_added_531normalized_predresp_rate.csv

### Outputs

/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/

RSG_SAGs_O2_consumption_2018-2019.csv

RSG_SAGs_O2_consumption_2018-2019_metagenome_GENs.csv


In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
import csv
from pathlib import Path
import seaborn as sns
sns.set()
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
from collections import defaultdict
import os.path as op
import glob

os.chdir('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/')
outdir=('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/')
matplotlib.__version__

'3.3.3'

In [2]:
O2=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption/GoM_mg_abundance_weighted_average_O2_consumption_per_cell.csv',
               index_col=0)
O2['avg_weighted_O2_consumption']=O2[['181030_weighted_avg_O2_consumption', '190402_weighted_avg_O2_consumption',
                                     '190709_weighted_avg_O2_consumption']].mean(axis=1)
O2_avg=O2[['genus', 'avg_weighted_O2_consumption']].copy()
metagenome_gens=O2_avg['genus'].to_list()
O2_avg

,genus,avg_weighted_O2_consumption
0,AAA164-E04,0.002050
1,AAA536-G10,0.003027
2,AG-337-I02,0.004461
3,AG-414-E02,0.002050
4,AG-422-B15,0.002432
...,...,...
163,Verrucomicrobiales,0.002050
164,Vibrio,0.002050
165,Vicingaceae,0.002050
166,Winogradskyella,0.002050


In [3]:
per_dayO2=O2[['181030_weighted_avg_O2_consumption', '190402_weighted_avg_O2_consumption',
                                     '190709_weighted_avg_O2_consumption', 'genus']].copy()
per_dayO2=per_dayO2.rename(columns={'181030_weighted_avg_O2_consumption':'20181030',
                                   '190402_weighted_avg_O2_consumption':'20190402',
                                   '190709_weighted_avg_O2_consumption':'20190709',})
per_day_O2_melt=pd.melt(per_dayO2, id_vars='genus', var_name='date', value_name='avg_fmol_O2_consumed')
per_day_O2_melt

,genus,date,avg_fmol_O2_consumed
0,AAA164-E04,20181030,0.002050
1,AAA536-G10,20181030,0.003056
2,AG-337-I02,20181030,0.009284
3,AG-414-E02,20181030,0.002050
4,AG-422-B15,20181030,0.002050
...,...,...,...
496,Verrucomicrobiales,20190709,0.001999
497,Vibrio,20190709,0.001999
498,Vicingaceae,20190709,0.001999
499,Winogradskyella,20190709,0.001999


In [4]:
abundance=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/GoM_Metagenome_analysis/GoM_metagenome_calculated_Genus_cells_per_ml.csv')
abundance['2018_2019_avg_cells_per_ml']=abundance[['20181030_genome_size_normalized_metagenome_cells/ml', '20190402_genome_size_normalized_metagenome_cells/ml',
                                                  '20190709_genome_size_normalized_metagenome_cells/ml']].mean(axis=1)
avg_cells_per_ml=abundance[['genus', '2018_2019_avg_cells_per_ml']].copy()
avg_cells_per_ml


,genus,2018_2019_avg_cells_per_ml
0,AAA164-E04,1047.825162
1,AAA536-G10,16346.796156
2,AG-337-I02,3978.439190
3,AG-414-E02,322.736406
4,AG-422-B15,7919.052060
...,...,...
163,Verrucomicrobiales,244.184201
164,Vibrio,706.453800
165,Vicingaceae,4235.203745
166,Winogradskyella,7776.721215


In [5]:
per_day_abundance=abundance[['20181030_genome_size_normalized_metagenome_cells/ml', '20190402_genome_size_normalized_metagenome_cells/ml',
                                                  '20190709_genome_size_normalized_metagenome_cells/ml', 'genus']].copy()

per_day_abundance=per_day_abundance.rename(columns={'20181030_genome_size_normalized_metagenome_cells/ml':'20181030',
                                                   '20190402_genome_size_normalized_metagenome_cells/ml': '20190402',
                                                   '20190709_genome_size_normalized_metagenome_cells/ml': '20190709'})
daily_abundance=pd.melt(per_day_abundance, id_vars='genus', var_name='date', value_name='cells_per_ml')
daily_abundance

,genus,date,cells_per_ml
0,AAA164-E04,20181030,1916.455808
1,AAA536-G10,20181030,16915.541760
2,AG-337-I02,20181030,9857.221878
3,AG-414-E02,20181030,438.765267
4,AG-422-B15,20181030,3313.158567
...,...,...,...
499,Verrucomicrobiales,20190709,280.890811
500,Vibrio,20190709,964.925624
501,Vicingaceae,20190709,5084.185705
502,Winogradskyella,20190709,10722.976925


In [6]:
average=avg_cells_per_ml.merge(O2_avg, how='left', on='genus')
average=average.drop_duplicates()
average.to_csv(outdir+'avg_cells_O2_consumption_per_GEN_2018-2019.csv')
average

,genus,2018_2019_avg_cells_per_ml,avg_weighted_O2_consumption
0,AAA164-E04,1047.825162,0.002050
1,AAA536-G10,16346.796156,0.003027
2,AG-337-I02,3978.439190,0.004461
3,AG-414-E02,322.736406,0.002050
4,AG-422-B15,7919.052060,0.002432
...,...,...,...
163,Verrucomicrobiales,244.184201,0.002050
164,Vibrio,706.453800,0.002050
165,Vicingaceae,4235.203745,0.002050
166,Winogradskyella,7776.721215,0.002050


In [7]:
per_day_melt=daily_abundance.merge(per_day_O2_melt, on=['genus','date'])
per_day_melt=per_day_melt.drop_duplicates()
per_day_melt.to_csv(outdir+'avg_O2_consumption_and_abundance_per_day_melt.csv')
per_day_melt

,genus,date,cells_per_ml,avg_fmol_O2_consumed
0,AAA164-E04,20181030,1916.455808,0.002050
1,AAA536-G10,20181030,16915.541760,0.003056
2,AG-337-I02,20181030,9857.221878,0.009284
3,AG-414-E02,20181030,438.765267,0.002050
4,AG-422-B15,20181030,3313.158567,0.002050
...,...,...,...,...
499,Verrucomicrobiales,20190709,280.890811,0.001999
500,Vibrio,20190709,964.925624,0.001999
501,Vicingaceae,20190709,5084.185705,0.001999
502,Winogradskyella,20190709,10722.976925,0.001999


In [8]:
SAG_data=pd.read_csv('/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/All_GoM_SAGs_1cell_20kb_decon_531normalized_predresp_rate_GTDBclass.csv')

dates=['181030', '190402','190709']

SAG_data=SAG_data.loc[SAG_data['Sample_collection_Date'].isin(dates)]
SAG_data=SAG_data.loc[SAG_data['gate']=='Respiring cells (RSG-positive)']
SAG_data=SAG_data.rename(columns={'GTDB_classification':'genus'})

Respiration=SAG_data[['name', 'genus', 'Predicted_respiration_rate_fmol_O2perhr', 'Sample_collection_Date']].copy()
#Respiration=Respiration.rename(columns={'group72.5':'gen'})
Respiration=Respiration.drop_duplicates()
Respiration.to_csv(outdir+'RSG_SAGs_O2_consumption_2018-2019.csv')

Respiration

,name,genus,Predicted_respiration_rate_fmol_O2perhr,Sample_collection_Date
1964,AH-545-A01,HIMB59,0.004897,181030
1965,AH-545-A02,UBA5951,0.045645,181030
1966,AH-545-A03,Unclassified,0.014962,181030
1967,AH-545-A04,Thioglobus_A,0.004827,181030
1968,AH-545-A05,Thioglobus,0.003989,181030
...,...,...,...,...
4728,AH-700-P10,Planktomarina,0.068309,190709
4729,AH-700-P11,Luminiphilus,0.003930,190709
4730,AH-700-P13,Planktomarina,0.029714,190709
4731,AH-700-P14,UBA8649,0.007228,190709


In [9]:
Respiration=Respiration.loc[Respiration['genus'].isin(metagenome_gens)]

Respiration.to_csv(outdir+'RSG_SAGs_O2_consumption_2018-2019_metagenome_GENs.csv')

Respiration

,name,genus,Predicted_respiration_rate_fmol_O2perhr,Sample_collection_Date
1964,AH-545-A01,HIMB59,0.004897,181030
1965,AH-545-A02,UBA5951,0.045645,181030
1966,AH-545-A03,Unclassified,0.014962,181030
1967,AH-545-A04,Thioglobus_A,0.004827,181030
1968,AH-545-A05,Thioglobus,0.003989,181030
...,...,...,...,...
4728,AH-700-P10,Planktomarina,0.068309,190709
4729,AH-700-P11,Luminiphilus,0.003930,190709
4730,AH-700-P13,Planktomarina,0.029714,190709
4731,AH-700-P14,UBA8649,0.007228,190709
